# Byte Pair Encoding Tokenization

## Introduction

**Byte-Pair Encoding (BPE)** is a tokenization method that was initially developed as an algorithm to **compress texts**. It was later used by OpenAI for tokenization when pretraining the GPT model. BPE is used by many Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa. This method allows the model to handle any word it encounters, even if it’s not in its vocabulary, by breaking it down into subwords it knows. This makes BPE very effective for NLP tasks.


In [ ]:
# Installing necessary library
!pip install datasets evaluate transformers[sentencepiece]

In [3]:
# Importing necessary library
from transformers import AutoTokenizer
from collections import defaultdict

In [4]:
# Creating a corpus
corpus = [
    "This is the Kalbe Digital Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [7]:
# Initialize tokenizer from pre-trained model
tokenizer = AutoTokenizer.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:
# Initializes a default dictionary where the default value for any key is int (0).
# It's used to store the frequency of each word in the corpus.
word_freqs = defaultdict(int)

# This loop iterates over each text in the corpus.
for text in corpus:
    # Tokenize the current text into words and their offsets.
    # The pre_tokenize_str method splits the input text into words and returns a list of tuples,
    # where each tuple contains a word and its offset in the original text.
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)

    # Extracts the words from the list of tuples (ignoring the offsets).
    new_words = [word for word, offset in words_with_offsets]

    # This loop iterates over each word in the current text.
    for word in new_words:
        # Increments the frequency count of the current word in the word_freqs dictionary.
        word_freqs[word] += 1

# Prints the word frequencies dictionary.
print(word_freqs)

defaultdict(<class 'int'>, {'This': 3, 'Ġis': 2, 'Ġthe': 1, 'ĠKalbe': 1, 'ĠDigital': 1, 'ĠCourse': 1, '.': 4, 'Ġchapter': 1, 'Ġabout': 1, 'Ġtokenization': 1, 'Ġsection': 1, 'Ġshows': 1, 'Ġseveral': 1, 'Ġtokenizer': 1, 'Ġalgorithms': 1, 'Hopefully': 1, ',': 1, 'Ġyou': 1, 'Ġwill': 1, 'Ġbe': 1, 'Ġable': 1, 'Ġto': 1, 'Ġunderstand': 1, 'Ġhow': 1, 'Ġthey': 1, 'Ġare': 1, 'Ġtrained': 1, 'Ġand': 1, 'Ġgenerate': 1, 'Ġtokens': 1})


In [9]:
# This line initializes an empty list named 'alphabet'.
# This list will be used to store the unique letters in the words.
alphabet = []

# This loop iterates over each word in the keys of the 'word_freqs' dictionary.
# The keys of the 'word_freqs' dictionary are the unique words in the corpus.
for word in word_freqs.keys():
    # This loop iterates over each letter in the current word.
    for letter in word:
        # This line checks if the current letter is not already in the 'alphabet' list.
        if letter not in alphabet:
            # If the current letter is not in the 'alphabet' list, it is appended to the list.
            alphabet.append(letter)

# This line sorts the 'alphabet' list in ascending order.
alphabet.sort()

# This line prints the sorted 'alphabet' list.
print(alphabet)

[',', '.', 'C', 'D', 'H', 'K', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ']


In [10]:
# Create a new vocabulary by adding an empty string as the first element and copying the alphabet
vocab = [""] + alphabet.copy()

# Create a dictionary of word splits, where each word is split into a list of its characters
splits = {word: [c for c in word] for word in word_freqs.keys()}

# Function to compute pair frequencies for consecutive characters in words
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

# Compute pair frequencies using the splits dictionary
pair_freqs = compute_pair_freqs(splits)

# Print the first 6 pairs and their frequencies
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


In [11]:
# Initialize variables to find the most frequent pair
best_pair = ""
max_freq = None

# Find the most frequent pair in the pair_freqs dictionary
for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

# Print the most frequent pair and its frequency
print(best_pair, max_freq)

# Define a manual merge for the pair ("Ġ", "t")
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

# Function to merge a specific pair in the splits dictionary
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

# Perform the manual merge for the pair ("Ġ", "t") in the splits dictionary
splits = merge_pair("Ġ", "t", splits)

# Print the resulting split for the word "Ġtrained"
print(splits["Ġtrained"])

('Ġ', 't') 7
['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


In [12]:
# Set the target vocabulary size
vocab_size = 50

# Continue merging pairs until the vocabulary size reaches the target
while len(vocab) < vocab_size:
    # Compute pair frequencies for the current splits
    pair_freqs = compute_pair_freqs(splits)

    # Find the most frequent pair
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq

    # Merge the best pair in the splits dictionary
    splits = merge_pair(*best_pair, splits)

    # Update the merges dictionary and vocabulary
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

# Print the final merges and vocabulary
print(merges)
print(vocab)

{('Ġ', 't'): 'Ġt', ('i', 's'): 'is', ('e', 'r'): 'er', ('Ġ', 'a'): 'Ġa', ('Ġt', 'o'): 'Ġto', ('e', 'n'): 'en', ('T', 'h'): 'Th', ('Th', 'is'): 'This', ('a', 'l'): 'al', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('Ġto', 'k'): 'Ġtok', ('Ġtok', 'en'): 'Ġtoken', ('n', 'd'): 'nd', ('Ġ', 'is'): 'Ġis', ('Ġt', 'h'): 'Ġth', ('Ġth', 'e'): 'Ġthe', ('b', 'e'): 'be'}
['', ',', '.', 'C', 'D', 'H', 'K', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'Ġ', 'Ġt', 'is', 'er', 'Ġa', 'Ġto', 'en', 'Th', 'This', 'al', 'ou', 'se', 'Ġtok', 'Ġtoken', 'nd', 'Ġis', 'Ġth', 'Ġthe', 'be']


In [15]:
def tokenize(text):
    # Use the pre-tokenizer to get initial tokenization results
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]

    # Split each pre-tokenized word into a list of characters
    splits = [[l for l in word] for word in pre_tokenized_text]

    # Apply merges based on the provided dictionary
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                # Merge consecutive characters if they match the pair
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    # Concatenate the split characters into the final tokenized result
    return sum(splits, [])

# Test the tokenize function with a sample input
tokenize("This is not a token.")

['This', 'Ġis', 'Ġ', 'n', 'o', 't', 'Ġa', 'Ġtoken', '.']